In [1]:
import matplotlib.dates as md
import pandas as pd
import numpy as np
import math
import datetime
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from matplotlib import rc
from patsy import *
plt.style.use('seaborn')
rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False
%matplotlib inline

-

In [2]:
# 데이터 설명 
# date - 날짜, temp - 온도, tour - 관광객 수, population - 제주도 인구수 ,humid - 상대 습도

In [1]:
jeju = pd.read_csv("datas/target_v3.csv")
jeju = jeju[["date","temp","population","humid"]]
jeju.tail()

,date,temp,population,humid
833,2020-05-14,18.7,670798.0,0.563
834,2020-05-15,19.9,670798.0,0.915
835,2020-05-16,17.6,670798.0,0.966
836,2020-05-17,17.1,670798.0,0.916
837,2020-05-18,19.2,670798.0,0.921


In [2]:
jeju_tour = pd.read_csv("datas/tour_stat.csv")
jeju_tour.tail(3)

,date,tour
835,2020-05-16,23009
836,2020-05-17,24394
837,2020-05-18,24001


### 문제 1 번 
- jeju 데이터와 jeju_tour 데이터 프레임을 합치시고 
- jeju_data 라고 데이터 프레임을 저장하세요!

In [5]:
# code
# '''여기에 쓰세요!'''

In [4]:
# 정답 
jeju_data = pd.concat([jeju,jeju_tour],axis =1)
jeju_data

,date,temp,population,humid,date,tour
0,2018-02-01,4.3,659189.0,0.601,2018-02-01,33144
1,2018-02-02,3.8,659234.0,0.553,2018-02-02,31549
2,2018-02-03,1.3,659278.0,0.673,2018-02-03,31348
3,2018-02-04,-1.0,659323.0,0.720,2018-02-04,26980
4,2018-02-05,-0.1,659368.0,0.712,2018-02-05,30188
...,...,...,...,...,...,...
833,2020-05-14,18.7,670798.0,0.563,2020-05-14,23327
834,2020-05-15,19.9,670798.0,0.915,2020-05-15,22135
835,2020-05-16,17.6,670798.0,0.966,2020-05-16,23009
836,2020-05-17,17.1,670798.0,0.916,2020-05-17,24394


In [7]:
# GRNI  수치는 GRDP/지역 인구를 의미한다.
# 제주도인 경우는 관광객 수를 GRDP로 친다고 하면
# jeju_data 프레임에서 위와 같은 수식을 이용한 GRNI의 '로그값'이 들어가는 열 을 추가하세요 ! (로그값!!!)

In [8]:
# code 는 여기에! 

In [5]:
# 정답  
jeju_data["GRNI"] = np.log(jeju_data["tour"]/jeju_data["population"])
jeju_data

,date,temp,population,humid,date,tour,GRNI
0,2018-02-01,4.3,659189.0,0.601,2018-02-01,33144,-2.990149
1,2018-02-02,3.8,659234.0,0.553,2018-02-02,31549,-3.039537
2,2018-02-03,1.3,659278.0,0.673,2018-02-03,31348,-3.045995
3,2018-02-04,-1.0,659323.0,0.720,2018-02-04,26980,-3.196118
4,2018-02-05,-0.1,659368.0,0.712,2018-02-05,30188,-3.083837
...,...,...,...,...,...,...,...
833,2020-05-14,18.7,670798.0,0.563,2020-05-14,23327,-3.358857
834,2020-05-15,19.9,670798.0,0.915,2020-05-15,22135,-3.411308
835,2020-05-16,17.6,670798.0,0.966,2020-05-16,23009,-3.372583
836,2020-05-17,17.1,670798.0,0.916,2020-05-17,24394,-3.314131


-

#### 2)  불쾌지수 추가
- 불쾌지수
    - 0.81 * 섭씨온도 +0.01 * 상대습도(%)（0.99*섭씨온도 - 14.3）+ 46.3



In [10]:
### 코드는 여기에!!### 
## 불쾌지수 컬럼을 추가한 jeju_data 프레임을 만들어 주세요
# 함수를 통해! 만들어서 해주세요 ~~ 
# 그냥하면 1점 감점 

In [6]:
# 정답 
#불쾌지수
def discomfort_index(temp, humid):
    discomfort = round(0.81 * temp + 0.01 * humid * (0.99 * temp - 14.3) + 46.3, 1)
    return discomfort
jeju_data["angry"] = discomfort_index(jeju_data["temp"], jeju_data["humid"])

jeju_data

,date,temp,population,humid,date,tour,GRNI,angry
0,2018-02-01,4.3,659189.0,0.601,2018-02-01,33144,-2.990149,49.7
1,2018-02-02,3.8,659234.0,0.553,2018-02-02,31549,-3.039537,49.3
2,2018-02-03,1.3,659278.0,0.673,2018-02-03,31348,-3.045995,47.3
3,2018-02-04,-1.0,659323.0,0.720,2018-02-04,26980,-3.196118,45.4
4,2018-02-05,-0.1,659368.0,0.712,2018-02-05,30188,-3.083837,46.1
...,...,...,...,...,...,...,...,...
833,2020-05-14,18.7,670798.0,0.563,2020-05-14,23327,-3.358857,61.5
834,2020-05-15,19.9,670798.0,0.915,2020-05-15,22135,-3.411308,62.5
835,2020-05-16,17.6,670798.0,0.966,2020-05-16,23009,-3.372583,60.6
836,2020-05-17,17.1,670798.0,0.916,2020-05-17,24394,-3.314131,60.2
